<a href="https://colab.research.google.com/github/alexmascension/ANMI/blob/main/notebook/T4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tema 5: Interpolación de funciones

In [ ]:
!pip install -r https://raw.githubusercontent.com/alexmascension/ANMI/main/requirements.txt

In [ ]:
from sympy import *
from sympy.matrices import Matrix as mat
from sympy.matrices import randMatrix
from sympy import symbols
import sympy

import numpy as np

from scipy.linalg import orth

from matplotlib import pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

In [ ]:
from anmi.genericas import norma_p_func, norma_inf_func

In [ ]:
x, y, z, a, lambda_ = symbols('x'), symbols('y'), symbols('z'), symbols('a'), symbols('lambda')

Dado un conjunto de puntos (nodos) $x_0, x_1, \cdots, x_m$ podemos crear un polinomio $p$ que aproxime la función $f$ y que cumpla que $p(x_i) = f(x_i)$ para todo nodo. Existen varias situaciones de interpolación:
* El número de nodos menos 1, $m-1$ es MAYOR que el máximo grado de polinomios, $n$.
* El número de nodos menos 1, $m-1$ es IGUAL que el máximo grado de polinomios, $n$.
* El número de nodos menos 1, $m-1$ es MENOR que el máximo grado de polinomios, $n$.

En este capítulo nos centraremos cuando $m - 1 = n$.

## Interpolación de Lagrange

En esta interpolación empleamos un conjunto de funciones $L = \{l_0, l_1, \cdots, l_n\}$ correspondiente a los $n+1$ nodos $\{x_i:i = 0, 1, \cdots, n\}$. Cada $l_i$ se define como

$$l_i(x) = \prod_{k = 0 \neq i}\frac{x - x_k}{x_i - x_k}$$

Y así el polinomio resultante es

$$p(x) = \sum_{i=0}^n l_i(x)f(x_i)$$

Donde $f(x_i)$ es el valor de la función $f$ en $x_i$.

In [ ]:
def polinomio_lagrange(x_vals, y_vals, var=x):
    lista_L, p, n = [], S(0), len(x_vals)
    for i in range(n):
        li = S(1)
        for k in range(n):
            if k != i:
                li *= (var - x_vals[k])/(x_vals[i] - x_vals[k])
        
        lista_L.append(li)
        p += y_vals[i] * li
    
    return p, lista_L

In [ ]:
f = E**(sin(4*x))
I = [0, 1]
x_vals = np.linspace(I[0], I[1], 4)
y_vals = [f.evalf(subs={x: S(i)}) for i in x_vals]

In [ ]:
p, lista_L = polinomio_lagrange(x_vals, y_vals)

In [ ]:
x_range = np.linspace(I[0], I[1], 100)
y_real = [f.subs(x, i) for i in x_range]
y_pol = [expand(p).subs(x, i) for i in x_range]


plt.plot(x_range, y_real, label=f'{f}')
plt.scatter(x_vals, y_vals, c="#800000")
plt.plot(x_range, y_pol, label='interpolacion', c="#bcbcbc")

plt.legend()

Ahora ploteamos cada una de las funciones

In [ ]:
x_range = np.linspace(I[0], I[1], 100)
y_pol = [expand(p).subs(x, i) for i in x_range]
y_real = [f.subs(x, i) for i in x_range]


for p_i, pl in enumerate(lista_L):
    y_l = [expand(pl).subs(x, i) * y_vals[p_i] for i in x_range]
    plt.plot(x_range, y_l, label=f'$l_{p_i}$')
    plt.scatter(x_vals[p_i], y_vals[p_i])

plt.plot(x_range, y_real, label=f'{f}')
plt.plot(x_range, y_pol, label='interpolacion', c="#bcbcbc")

plt.legend(bbox_to_anchor=(1.05, 0.5))

## Método de Newton
El método de Lagrange, aunque da un polinomio único, tiene una serie de problemas. En primer lugar, aunque computacionalmente no parezca complejo, el cálculo de coeficientes es tedioso y poco práctico para ciertas funciones. Además, si queremos añadir un nuevo nodo, tenemos que recalcular todas las funciones de Lagrange para el polinomio nuevo, lo cual lo vuelve bastante impráctico. 

Con el método de Newton solventamos las problemáticas estableciendo un método iterativo que incorpora un nodo por vez.

En el método se aplican los siguiqntes cálculos:

$$
\begin{matrix}
x_0 & \color{blue}{f(x_0)} & & \\
  & & \color{blue}{\frac{f(x_1) - f(x_0)}{x_1 - x_0}} & \\
x_1   & f(x_1) & & \color{blue}{\frac{\frac{f(x_2) - f(x_1)}{x_2 - x_1} - \frac{f(x_1) - f(x_0)}{x_1 - x_0}}{x_2 - x_0}}\\
    & & \frac{f(x_2) - f(x_1)}{x_2 - x_1}& \\
x_2     &f(x_2) & & \\
\vdots     & \vdots & & \\
x_{n-1}     &f(x_{n-1}) & & \\
& & \frac{f(x_n) - f(x_{n-1})}{x_n - x_{n-1}} & \\
x_n     &f(x_n) & & \\
\end{matrix}
$$

O de manera compacta:
$$
\begin{matrix}
x_0 & \color{blue}{f(x_0)} & & \\
  & & \color{blue}{f[x_0, x_1]} & \\
x_1   & f(x_1) & & \color{blue}{f[x_0, x_1, x_2]}\\
    & & \color{blue}{f[x_1, x_2]}& \\
x_2     &f(x_2) & & \\
\vdots     & \vdots & & \\
x_{n-1}     &f(x_{n-1}) & & \\
& & \color{blue}{f[x_{n-1}, x_n]} & \\
x_n     &f(x_n) & & \\
\end{matrix}
$$ 
Con
$$f[x_a, x_{a+1}, \cdots, x_j] = \frac{f[x_{a+1}, x_{a + 2}, \cdots, x_j] - f[x_{a}, x_{a+1}, \cdots, x_{j-1}]}{x_j - x_a}$$
Si denominamos $a_0, a_1, \cdots, a_n$ a los monomios en azul ($a_0 = f(x_0)$, $a_1 = \frac{f(x_1) - f(x_0)}{x_1 - x_0}$, etc.) entonces el polinomio de Newton se construye como:
$$p_n(x) = a_0 + a_1(x - x_0) + a_2(x - x_0)(x - x_1) + \cdots + a_n(x - x_0)(x - x_1)\cdots(x - x_{n-1})$$


In [ ]:
def polinomio_newton(x_vals, y_vals, var=x):
    matriz_coeffs = zeros(len(y_vals), len(y_vals))
    
    for i in range(len(y_vals)): # Asignamos la primera columna como f(x)
        matriz_coeffs[i,0] = y_vals[i]
    
    for col in range(1, len(y_vals)):
        for row in range(len(y_vals) - col):
            num = matriz_coeffs[row + 1, col - 1] - matriz_coeffs[row, col - 1]
            den = x_vals[row + col] - x_vals[row]
            
            matriz_coeffs[row, col] = S(num) / S(den)
    
    p = matriz_coeffs[0, 0]
    for col in range(1, len(y_vals)):
        p_col = S(1)
        for i in range(0, col):
            p_col *= (var - x_vals[i])
        
        p += p_col * matriz_coeffs[0, col]
    
    return p, matriz_coeffs

In [ ]:
p, mat = polinomio_newton([S(i) for i in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]], [S(i) for i in [0.7, 0.8, 1, 1.15, 1.25, 1.3]], x)
mat

In [ ]:
p

In [ ]:
f = E**(sin(4*x))
I = [0, 1]
x_vals = np.linspace(I[0], I[1], 4)
y_vals = [f.evalf(subs={x: S(i)}) for i in x_vals]

In [ ]:
p, mat = polinomio_newton(x_vals, y_vals, x)
p